In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchsummary import summary
from tqdm import tqdm

In [2]:
from utils import get_device,set_seed
from trainer import Trainer
from tester import Tester
from dataloader import Cifar10D
from transformations import std_transforms,train_transforms

Device selected cuda


In [3]:
set_seed(mix_precision=True)

In [4]:
device = get_device()
print(device)

cuda


In [5]:
class Model1(nn.Module):
    def __init__(self,norm_name:str,dropout_rate:float):
        super(Model1,self).__init__()

        self.P = nn.MaxPool2d(2,2)
        self.dropout_rate = dropout_rate
        
        self.C1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )
        self.C2 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )
        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=12,kernel_size=1,stride=1,padding=1),
        )

        self.C4 = nn.Sequential(
            nn.Conv2d(in_channels=12,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )
        self.C5 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )
        self.C6 = nn.Sequential(
            nn.Conv2d(in_channels=12,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        ) 
        self.c7 = nn.Sequential(
           nn.Conv2d(in_channels=32,out_channels=16,kernel_size=1,stride=1,padding=1)
        ) 

        self.C8 = nn.Sequential(
             nn.Conv2d(in_channels=16,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )
        self.C9 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        ) 
        self.C10 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        ) 

        self.gap = nn.AdaptiveAvgPool2d(output_size=1)
        
        self.c11 = nn.Conv2d(in_channels=64,out_channels=10,kernel_size=1,stride=1)
    
    def forward(self,x):
        x1 = self.C1(x); x2 = self.C2(x1)
        x = x1+x2     
        
        x = self.c3(x); 
        x = self.P(x)  
        
        x4 = self.C5(self.C4(x)) ; x5 = self.C6(x)
        x = x4+x5
        
        x = self.c7(x)
        x = self.P(x)
        
        x6 = self.C9(self.C8(x));  x7=self.C10(x)
        x = x6+x7
        x = self.gap(x)
        x = self.c11(x)
        return F.log_softmax(x.view(-1,10),dim=1) 


In [6]:
class Module0(nn.Module):
    def __init__(self,norm_name:str,dropout_rate:float=0.01):
        super(Module0,self).__init__()

        self.dropout_rate = dropout_rate
        self.norm_name    = norm_name

        self.conv1 = self.ConvLayer(in_=3,out_=8,k=3,s=1,p=1).extend(
                            self.ConvLayer(in_=8,out_=10,k=3,s=1,p=1)                
                        )
        
        self.trans1 = nn.Sequential(
                            nn.MaxPool2d(2,2),
                            nn.Conv2d(in_channels=10,out_channels=8,kernel_size=1,stride=1,bias=False)
                    )

        self.conv2 = self.ConvLayer(in_=8,out_=8,k=3,s=1,p=1).extend(
                            self.ConvLayer(in_=8,out_=16,k=3,s=1,p=1)
                    )
        
        self.trans2 = nn.Sequential(
                            nn.MaxPool2d(2,2),
                            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=1,stride=1,bias=False)
                    )
        
        self.conv3 = self.ConvLayer(in_=16,out_=32,k=3,s=1,p=1).extend(
                        self.ConvLayer(in_=32,out_=40,k=3,s=1,p=1)
                        ).extend(
                                self.ConvLayer(in_=40,out_=64,k=3,s=1,p=1)
                        )
        self.gap = nn.AdaptiveAvgPool2d(output_size=1)
        self.conv4 = nn.Conv2d(in_channels=64,out_channels=10,kernel_size=1,bias=False)
        
    def ConvLayer(
                self,
                in_:int,
                out_:int,
                k:int,
                s:int,
                p:int,
                b=False
    )->torch.nn.modules.container.Sequential:
        return nn.Sequential(
            nn.Conv2d(in_channels=in_,out_channels=out_,kernel_size=k,stride=s,padding=p,bias=b),
            self.get_norm(out_=out_),
            nn.ReLU(inplace=True),
            nn.Dropout2d(self.dropout_rate)
        )

    def get_norm(self,out_):
        if self.norm_name=='bn':
            return nn.BatchNorm2d(num_features=out_)
        else:
            raise ValueError('Pass proper normalisation')

    def forward(self,x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.trans2(x)
        x = self.conv3(x)
        x = self.gap(x)
        x = self.conv4(x)
        logit = F.log_softmax(x.view(-1,10),dim=1) 
        return logit

In [7]:
cifar10 = Cifar10D(batch_size=128,is_cuda_available=True)

train_loader = cifar10.get_loader(root='../data/',download=False,transform=train_transforms, train=True)
test_loader = cifar10.get_loader(root='../data/',download=False,transform=std_transforms, train=False)

In [8]:
# model = Module0(norm_name='bn',dropout_rate=0.1).to(device) #train=50.37 / test=59.57
model = Model1(norm_name='bn',dropout_rate=0.2).to(device)

In [9]:
# Data to plot accuracy and loss graphs (INIT)
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()


def train(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # print(pred.shape)
    # print(target.shape)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc.append(100*correct/processed)
  train_losses.append(train_loss/len(train_loader))

def test(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()  # sum up batch loss

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)
    
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss


In [10]:
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
         Dropout2d-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           2,320
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
         Dropout2d-8           [-1, 16, 32, 32]               0
            Conv2d-9           [-1, 12, 34, 34]             204
        MaxPool2d-10           [-1, 12, 17, 17]               0
           Conv2d-11           [-1, 32, 17, 17]           3,488
      BatchNorm2d-12           [-1, 32, 17, 17]              64
             ReLU-13           [-1, 32, 17, 17]               0
        Dropout2d-14           [-1, 32,

In [ ]:
%%time

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1, verbose=True)
# New Line
criterion =  F.nll_loss
num_epochs =20

for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train(model, device, train_loader, optimizer, criterion)
  test(model, device, test_loader, criterion)
  scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-01.
Epoch 1


Train: Loss=1.6326 Batch_id=390 Accuracy=26.89: 100%|████████████████████████████████████████████████████████████████████| 391/391 [00:40<00:00,  9.66it/s]


Test set: Average loss: 1.6281, Accuracy: 3684/10000 (36.8400%)

Adjusting learning rate of group 0 to 1.0000e-01.
Epoch 2


Train: Loss=1.4587 Batch_id=390 Accuracy=38.98: 100%|████████████████████████████████████████████████████████████████████| 391/391 [00:39<00:00,  9.96it/s]


Test set: Average loss: 1.4190, Accuracy: 4722/10000 (47.2200%)

Adjusting learning rate of group 0 to 1.0000e-01.
Epoch 3


...

Train: Loss=1.3693 Batch_id=390 Accuracy=57.42: 100%|████████████████████████████████████████████████████████████████████| 391/391 [00:46<00:00,  8.43it/s]


Test set: Average loss: 0.9768, Accuracy: 6548/10000 (65.4800%)

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 14


Train: Loss=1.2822 Batch_id=390 Accuracy=57.25: 100%|████████████████████████████████████████████████████████████████████| 391/391 [00:44<00:00,  8.72it/s]


Test set: Average loss: 0.9754, Accuracy: 6550/10000 (65.5000%)

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 15


  0%|                                                                                                                              | 0/391 [00:00<?, ?it/s]

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(15,10))

axs[0, 0].plot(train_losses)
axs[0, 0].set_title("Training Loss")

axs[1, 0].plot(train_acc)
axs[1, 0].set_title("Training Accuracy")

axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")

axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")
plt.show()

In [ ]:
from utils import show_egs

In [ ]:
show_egs(train_loader,figsize=(15,10))

In [ ]:
def get_misclassified_images(model,test_loader):
    model.eval()
    images = []
    predictions  = []
    labels = []

    with torch.no_grad():
        for inputs,targets in test_loader:
            inputs  = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            _,pred = torch.max(outputs,1)

            for i in range(len(pred)):
                if pred[i]!=targets[i]:
                    images.append(inputs[i])
                    predictions.append(pred[i])
                    labels.append(targets[i])
    return images,predictions,labels

In [ ]:
imgs, preds, lbls = get_misclassified_images(model,test_loader)

In [ ]:
def show_misclassified_images( images, predictions, labels, classes):
    assert len(images) == len(predictions) == len(labels)

    fig = plt.figure(figsize=(20, 10))
    for i in range(len(images)):
        sub = fig.add_subplot(len(images) // 5, 5, i + 1)
        image = images[i]
        image = image.T
        npimg = image.cpu().numpy().squeeze()
        plt.imshow(npimg, cmap="gray")
        predicted = classes[predictions[i]]
        correct = classes[labels[i]]
        sub.set_title(
            "Correct class: {}\nPredicted class: {}".format(correct, predicted)
        )
    plt.tight_layout()

In [ ]:
show_misclassified_images(images=imgs[:20],predictions=preds[:20],labels=lbls[:20],classes=cifar10.classes)